In [2]:
#The main Jupyter notebook for running our simulation. For now we can also use this notebook to test out code segments

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import copy
import CSJModel as csj

In [3]:
#Model Parameters
length_of_growing_season = 150
n_seasons_per_trial = 10
n_trials = 10000
plot_capacity = 1000
initial_soil_sat = 0.5
base_soil_sat_loss_rate = 0.05
leaf_transpiration_rate = 0.01
root_retention_rate = 0.01

In [ ]:
#This line creates a new plant species and saves it in the test variable. This species has a base growth rate of 1%/day, likes 50%
#soil saturation, and has saturation tolerance parameter of 10, and is starting with 10 biomass units
#In order to get the plant's actual growth rate, we call the .growth_rate() method and pass in the current level of soil saturation

grass = Plant('grass', 0.01, 0.5, 10, 10, 0.3)
rose = Plant('rose', 0.02, 0.7, 15, 10, 0.5)

In [5]:
#Set up a function for running a single trial that returns summary results from the trial.

def run_trial(plot_list):
    daily_list = []
    for season in range(n_seasons_per_trial):
        for time in range(length_of_growing_season):
            
            #FIGURE OUT WHAT THE WEATHER WILL BE HERE
            if random.randint(0, 100) > 94:
                rain = True
            else:
                rain = False
            
            for plot in plot_list:
                datapoint = {'season': season, 'time': time, 'plot': plot.plot_id}
                
                #DEAL WITH THE WEATHER HERE
                if rain:
                    plot.wet(0.8)
                else:
                    plot.dry()
                datapoint['soil_sat'] = test_plot.soil_saturation
                
                #Let the Plants Grow
                for plant in plot.species_list:
                    other_biomass = 0
                    for plant2 in plot.species_list:
                        if plant2 <> plant:
                            other_biomass += plant2.biomass
                    plant.biomass += plant.growth_rate(plot.soil_saturation) * plant.biomass * (1 - (plant.biomass/(plot.capacity - np.log(1 + other_biomass))))
                    datapoint[plant.name + '_biomass'] = plant.biomass
            
            
                daily_list.append(datapoint)
        df = pd.DataFrame.from_records(daily_list, index=['season','time','plot'])
        
    return df
        

In [ ]:
#DEFINE PLANTS

#DEFINE PLOTS

plot_list = [
                #PLOT_1,
                #PLOT_2,
                #...
                #PLOT_N
]

#Outer Monte Carlo-Style loop
trial_dfs = []
for trial in range(n_trials):
    trial_dfs.append(run_trial(copy.deepcopy(plot_list)))